<a href="https://colab.research.google.com/github/albertomariapepe/Learning-Rotations/blob/main/LearningRotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/pygae/clifford.git@master

  Cloning https://github.com/pygae/clifford.git (to revision master) to /tmp/pip-req-build-z2rdw9c0
  Running command git clone -q https://github.com/pygae/clifford.git /tmp/pip-req-build-z2rdw9c0
     |████████████████████████████████| 81kB 10.6MB/s 
  Created wheel for clifford: filename=clifford-1.4.0.dev0-cp37-none-any.whl size=156870 sha256=339adeb5b448e94b3ae288c49c862e5c27ea4f3a87e5478d46b6ac87d5ab3f93
  Stored in directory: /tmp/pip-ephem-wheel-cache-a7gndwj_/wheels/81/d0/f1/6422c3ef192b296a0055bcf7257d582555aaad0e9b8e810689
Successfully built clifford


In [ ]:
!pip install tensorflow_graphics

In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np
from clifford.g3c import *
from clifford.tools.g3c import *
from clifford.tools.g3c.rotor_parameterisation import *
from tensorflow.keras.layers import Input, Dense, LeakyReLU
from keras import backend as K
from sklearn.model_selection import train_test_split
from math import acos
import tensorflow as tf
from matplotlib import pyplot as plt
import keras
from math import e, pi
import tensorflow_graphics as tfg
import tensorflow_graphics.geometry.transformation as tfg_transformation


#GENERATING THE RANDOM DATASET
tot = int(1e5)
i = 0


#r = []
#ee = []
#a = []
#q = []
#b = []
#c = []
'''
for i in range(0,tot):
    r = np.append(r, R.random(random_state = i).as_matrix())
    #ee = np.append(ee, R.random(random_state = i).as_euler('zxy'))
    #a = np.append(a, R.random(random_state = i).as_rotvec())
    #q = np.append(q, R.random(random_state = i).as_quat())

    Q = R.random(random_state = i).as_quat()
    #C = (Q - 1)/(Q+1)
    #print(C)
    #c = np.append(c, C/np.linalg.norm(C))
    Rot = Q[0] + Q[2]*e13 +Q[2]*e23 + Q[3]*e12
    B = -2*general_logarithm(Rot)
    #b = np.append(b, [B[6], B[7], B[10]])
'''

#np.save('R.npy', r)
#np.save('E.npy', ee)
#np.save('A.npy', a)
#np.save('Q.npy', q)
#np.save('C.npy', c)




r = np.load('R.npy')
a = np.load('A.npy')
ee = np.load('E.npy')
q = np.load('Q.npy')
b = np.load('B.npy')


r = np.reshape(r, [tot, 9])   
ee = np.reshape(ee, [tot, 3])
a = np.reshape(a, [tot, 3])
q = np.reshape(q, [tot, 4])
b = np.reshape(b, [tot, 3])
#c = np.reshape(c, [tot, 4])

#Train - Test Split
r_train, r_test = train_test_split(r, test_size=0.33, shuffle=False)
e_train, e_test = train_test_split(ee, test_size=0.33, shuffle=False)
a_train, a_test = train_test_split(a, test_size=0.33, shuffle=False)
q_train, q_test = train_test_split(q, test_size=0.33, shuffle=False)
b_train, b_te4st = train_test_split(b, test_size=0.33, shuffle=False)
#c_train, c_test = train_test_split(c, test_size=0.33, shuffle=False)


TRAIN = q_train
TEST = q_test
out_size = 4 

nb_epoch = 100
batch_size = 64

'''
def l2_loss(y_true, y_pred):
    y_true = tfg_transformation.rotation_matrix_3d.from_quaternion(y_true)
    y_pred = tfg_transformation.rotation_matrix_3d.from_quaternion(y_pred)
    l2 = tf.nn.l2_loss(y_true - y_pred)
    return l2
'''
def l2_loss(y_true, y_pred):

    y_true = tfg_transformation.rotation_matrix_3d.from_quaternion(y_true)
    y_pred = tfg_transformation.rotation_matrix_3d.from_quaternion(y_pred)

    l2 = tf.nn.l2_loss(y_true - y_pred)
    return l2




r_input = Input(shape=(9))
x = Dense(128)(r_input)
x = LeakyReLU(alpha=0.2)(x)
x = Dense(128)(x)
x = LeakyReLU(alpha=0.2)(x)
x = Dense(128)(x)
x = LeakyReLU(alpha=0.2)(x)
out = Dense(out_size)(x)
print(out)


model = keras.Model(r_input,  out)
model.summary()
model.compile(loss=l2_loss, optimizer="adam")
#es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model_train = model.fit(x = r_train, y = TRAIN, 
                        validation_split=0.3,
                        epochs=nb_epoch,
                        verbose=1,
                        batch_size=batch_size)



loss = model_train.history['loss']
val_loss = model_train.history['val_loss']
epochs = range(nb_epoch)
plt.figure()
plt.plot(epochs, loss, 'b-', label='Training loss')
plt.plot(epochs, val_loss, 'r-', label='Validation loss')
plt.title('Training loss')
plt.legend()
plt.show()

predicted = model.predict(r_test)

M = []
Langle = []

for i in range(0,int(tot*0.33)):
    
    #C_predicted = (1 + predicted[i])/(1 - predicted[i])
    r_predicted = R.from_quat(predicted[i])
    y_pred = r_predicted.as_matrix()

    #C_TEST = (1 + TEST[i])/(1 - TEST[i])
    r_real = R.from_quat(C_TEST[i])
    y_real = r_real.as_matrix()
    
    M= np.matmul(y_real, np.linalg.inv(y_pred))
    cosine = (M[0,0] + M[1,1] + M[2,2] - 1)/2
    if cosine > 1:
        cosine = 1
    Langle = np.append(Langle, acos(cosine))

'''
from math import e

for i in range(0,int(tot*0.33)):
    B = predicted[i][0]*e12 +predicted[i][1]*e13 + predicted[i][2]*e23
    Rot = e**(-B/2)
    pred = [Rot[0], Rot[10], Rot[6], Rot[7]]
    r_predicted = R.from_quat(pred)
    y_pred = r_predicted.as_matrix()

    B = TEST[i][0]*e12 +TEST[i][1]*e13 + TEST[i][2]*e23
    Rot = e**(-B/2)
    real = [Rot[0], Rot[10], Rot[6], Rot[7]]
    r_real = R.from_quat(real)
    y_real = r_real.as_matrix()
    
    M= np.matmul(y_real, np.linalg.inv(y_pred))
    cosine = (M[0,0] + M[1,1] + M[2,2] - 1)/2
    if cosine > 1:
        cosine = 1
    Langle = np.append(Langle, acos(cosine))
'''

print(np.max(Langle)*180/pi)
print(np.average(Langle)*180/pi)
print(np.std(Langle)*180/pi)

#np.save('B_loss.npy', loss)
#np.save('B_val_loss.npy', val_loss)
#np.save('B_langle.npy', Langle)



KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), name='dense_7/BiasAdd:0', description="created by layer 'dense_7'")
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 34,691
Trainable params: 34,691
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


ValueError: ignored

In [ ]:
from math import cos, sin, pi
import numpy as np
theta = 30*pi/180
deltat = 1*pi/180
L = 1

r = [cos(theta), 0, sin(theta), 0, cos(theta), 0, -sin(theta), 0, 1]
r = np.reshape(r, [3, 3])
print(r)

from scipy.spatial.transform import Rotation as R
r_matrix = R.from_matrix(r)
q = r_matrix.as_quat()

Q = R.from_quat(q)

print(q)
print('---')
#R = Q.as_matrix()
#print(R)

for L in range(0,10):
    rd = [cos(theta+L*deltat), 0, sin(theta+L*deltat), 0, cos(theta+L*deltat), 0, -sin(theta+L*deltat), 0, 1]
    rd = np.reshape(rd, [3, 3])
    r_matrix = R.from_matrix(rd)
    q = r_matrix.as_quat()
    print(q)


[[ 0.8660254  0.         0.5      ]
 [ 0.         0.8660254  0.       ]
 [-0.5        0.         1.       ]]
[0.         0.25881905 0.         0.96592583]
---
[0.         0.25881905 0.         0.96592583]
[0.         0.26723838 0.         0.96363045]
[0.         0.27563736 0.         0.9612617 ]
[0.         0.28401534 0.         0.95881973]
[0.         0.2923717  0.         0.95630476]
[0.         0.3007058  0.         0.95371695]
[0.         0.30901699 0.         0.95105652]
[0.         0.31730466 0.         0.94832366]
[0.         0.32556815 0.         0.94551858]
[0.         0.33380686 0.         0.94264149]


In [ ]:
!zip -r /content/ /content

from google.colab import files
files.download("/content/.zip")

In [ ]:
cos(theta/2)

0.9659258262890683